In [1]:
import os
import sys

sys.path.insert(0,'/home/jupyter/git-repos/substitution-offline-simulation/') # add repository path to identify local modules
from src.helper_functions import *
from src.database_handler import DatabaseHandler

In [2]:
from src.generate_simulations import *

Step 1/5: Grabbing the newest bubblesort dict...
Step 2/5: Create SKU dictionaries for VCD, RCTS, Shipping, Shipping Savings for 5 and 20 region models
Step 3/5: Generate top 192 Bubblesort candidates for each class
Step 4/5: Create out-of-region anchor sku substitutes dictionaries for generated bubblesort skus
Fetching anchor-substitutes from 5 region offline pipeline...
Fetching anchor-substitutes from 20 region offline pipeline...
step 5/5: Loading functions for different test lanes, geosort, substitution functions


In [3]:
df_full_geo = run_simulations(arms = ARMS, max_pos=MAX_POS,max_subs=MAX_SUBS)

In [5]:
#print(df_full_geo.head())
print(df_full_geo.head())
#df_full_geo.to_pickle("df_full_geo.pkl")

                pos   sort_scores    vcd_values  shipping_values  \
count  34560.000000  34560.000000  34560.000000     34560.000000   
mean      96.500000      0.293846    120.677383        37.607824   
std       55.425676      0.284612    117.189836        46.929033   
min        1.000000     -0.438421   -275.030000         4.771622   
25%       48.750000      0.093372     45.700000        12.295657   
50%       96.500000      0.324403     80.330000        21.690000   
75%      144.250000      0.582376    158.020000        44.732500   
max      192.000000      1.000000    753.739431       527.380000   

       shipping_savings_values    cvr_scores  normalized_cvr_scores  \
count             34560.000000  34560.000000           34560.000000   
mean                  0.417074      0.004328               0.531288   
std                  21.240827      0.001785               0.149229   
min                -192.330000      0.000550               0.084566   
25%                  -1.650000  

**Note: Dataframe 'df_full_geo' now contains simulated default pageloads for the chosen configurations in {local_path}/substitution-offline-simulation/constants.py**

Next steps (optional): 
- Add information about if price, reviews, ratings, sku positioning, counterfactual model estimations to the simulations dataframe
- Check impression weighted analysis between two arms for each region Ex: {20 Region bubblesort (no alpha) - Substitution (alpha)}
- Check order-weighted average between two arms across all regions

## Add well-positioned column if needed for the analysis

In [6]:
## optional - skip if not needed
anchor_mapdf = pd.DataFrame.from_dict(anchor_map_20region, orient = 'index').reset_index()
anchor_map20_long = pd.melt(anchor_mapdf, id_vars='index')
anchor_map20_long.columns = ['geo', 'sku', 'recskus']
anchor_map20_long['region_model'] = '20region'
anchor_map20_long['well_positioned'] = 0
anchor_map20_long.loc[anchor_map20_long.recskus.isna(),'well_positioned'] = 1


anchor_mapdf = pd.DataFrame.from_dict(anchor_map_5region, orient = 'index').reset_index()
anchor_map5_long = pd.melt(anchor_mapdf, id_vars='index')
anchor_map5_long.columns = ['geo', 'sku', 'recskus']
anchor_map5_long['region_model'] = '5region'
anchor_map5_long['well_positioned'] = 0
anchor_map5_long.loc[anchor_map5_long.recskus.isna(),'well_positioned'] = 1


df_full_geo_20reg = pd.merge(df_full_geo[df_full_geo.arm != 'control'], anchor_map20_long, on = ['geo','sku'], how = 'left')
df_full_geo_20reg.loc[df_full_geo_20reg.well_positioned.isna(), "well_positioned"] = 1
df_full_geo_20reg.loc[df_full_geo_20reg.well_positioned.isna(), "region_model"] = "20region"

df_full_geo_5reg = pd.merge(df_full_geo[df_full_geo.arm == 'control'], anchor_map5_long, on = ['geo','sku'], how = 'left')
df_full_geo_5reg.loc[df_full_geo_5reg.well_positioned.isna(), "well_positioned"] = 1


df_full_geo = df_full_geo_5reg.append(df_full_geo_20reg).reset_index(drop=True)

In [7]:
# Get additional metrics avg price, rating, reviews on all simulations dataframe
df_full_geo_metrics = get_additional_metrics(df_full_geo, CLASSES_COVERED)

In [8]:
print(df_full_geo.head())

   pos       sku  sort_scores  vcd_values  shipping_values  \
0    1  OPCO7411     0.670186   22.355685        21.498818   
1    2  PBVR5363     0.644301   14.619063         7.999539   
2    3  BDJK1193     0.608410   58.681861        10.964452   
3    4  MCCY4357     0.612760   47.572268         9.944685   
4    5  FSTI1732     0.816176   24.117307        16.608900   

   shipping_savings_values  cvr_scores  normalized_cvr_scores  \
0                -9.519491    0.006162               0.670186   
1                 0.125153    0.005924               0.644301   
2                 2.268656    0.005594               0.608410   
3                 0.748651    0.005634               0.612760   
4                 0.000000    0.007504               0.816176   

   normalized_scores alpha geo  source_type      arm  clid  sample recskus  \
0           0.105811   0.0   1       hedwig  control   167       0     NaN   
1           0.000000   0.0   1       hedwig  control   167       0     NaN   
2 

## Counterfactual model predictions

We can use counterfactual model to predict the probability of customer conversion in addition to bubblesort scores used to represent conversion score. Please refer to the MVP 
[document](https://docs.google.com/document/d/14c5YCYPjugfdtosP2dhPP6SAVCV3YbSlpS-1doxf4DU/edit#) 

In [ ]:
## Add counterfactual model predictions

from sub_v15_cfmodel.scorer import CounterfactualScorer
# Instantiate purchase model
# User older VCD table for 5 region model
cvr_scorer_5region = CounterfactualScorer(
    rcts_table=VCD_TABLE_NAME, 
    ship_dist_table=SHIP_DIST_NAME_5REGION
)
# Use up to date VCD table for 20 region model
cvr_scorer_20region = CounterfactualScorer(
    rcts_table=VCD_TABLE_NAME_20REGION,
    ship_dist_table=SHIP_DIST_NAME_20REGION
)

# Get conversion rate probabilities
# These don't depend on geo, will be the same with either model
# These will need to be multiplied by the number of pageloads to get total orders
# Note that we will need to rename a couple of columns to match how the model was trained
# Recent date for getting features
event_date = '2022-07-12'

sku_df = df_full_geo_metrics[['pos', 'sku', 'clid']].rename(columns={'pos': 'skupagerank', 'clid': 'classid'})
results_df = cvr_scorer_5region.predict(sku_df, event_date)
df_full_geo_metrics['cf_score'] = results_df['cf_score']

In [ ]:
# with open('df_full_geo_metrics_7_15_2022.pkl', 'wb') as file:
#     pickle.dump(df_full_geo_metrics, file)

### Arguments for creating plots {Control (group1) - Test (group2)}

1. df_full_geo_metrics (DataFrame): output from get_additional_metrics function
2. control_alpha (float): Pick an alpha value of interest between 0.0 - 1.0
3. control_arm (String): Pick an arm of interest between control, variation2, variation3, reg20_bubblesort
2. test_alpha (float): Pick an alpha value of interest between 0.0 - 1.0
3. test_arm (String): Pick an arm of interest between control, variation2, variation3, reg20_bubblesort
4. estimation (String): Choose between two averaging methods while aggregating between position cutoffs
       - "simple" averages between SKU metrics across chosen classes for a given geo on skus between 1-12, 12-48, 48-96, 96-144, 144-192
       - "rolling" averages between SKU metrics across chosen classes for a given geo on skus between 1-12, 1-48, 1-96, 1-144, 1-192

# Impression weighted analysis {20 Region bubblesort (no alpha) - Substitution (alpha)}

- We have analysis functions that assigns different weights on skus based on their classses. We will be fetching visits(impressions) on all the classes for calculating the difference between weight proportioned Alpha = 0.0 group and Alpha > 0.0 group

In [9]:
# Daterange to pull impressions and orders for weighthed averages. Set 
start_date = "2022-05-01"
end_date = "2022-06-01"

db_handler = DatabaseHandler()

In [11]:
# pull impressions by class, pos, region for a timerange (change dates as needed for your analysis)
# please change the time period to fit to your analysis
df_visits = db_handler.get_impressions_by_pos_plus_clid_region(
    CLASSES_COVERED,
    SOID,
    #metrics,
    start_date,
    end_date,
    GEOS,
    CUSTOMER_REGION_MAPPING['20region'],
    SUPPLIER_REGION_MAPPING['20region'],
)

df_visits.to_pickle("df_visits.pkl")
# with open('df_visits_06_15_2022.pkl', 'wb') as file:
#     pickle.dump(df_visits, file)

In [ ]:
with open('df_visits_06_15_2022.pkl', 'rb') as file:
    df_visits = pickle.load(file)

In [ ]:
metrics

In [ ]:
df_full_geo_metrics

### Arguments for creating plots {Control (group1) - Test (group2)}

1. df_full_geo_metrics (DataFrame): output from get_additional_metrics function
2. control_alpha (float): Pick an alpha value of interest between 0.0 - 1.0
3. control_arm (String): Pick an arm of interest between control, variation2, variation3, reg20_bubblesort
2. test_alpha (float): Pick an alpha value of interest between 0.0 - 1.0
3. test_arm (String): Pick an arm of interest between control, variation2, variation3, reg20_bubblesort
4. estimation (String): Choose between two averaging methods while aggregating between position cutoffs
       - "simple" averages between SKU metrics across chosen classes for a given geo on skus between 1-12, 12-48, 48-96, 96-144, 144-192
       - "rolling" averages between SKU metrics across chosen classes for a given geo on skus between 1-12, 1-48, 1-96, 1-144, 1-192

In [ ]:
control_alpha = 0.0
control_arm = "reg20_geosort"
test_alpha = 0.5
test_arm = "variationA"
estimation = "rolling"
plot_comparison(
    df_full_geo_metrics, control_alpha, test_alpha, control_arm, test_arm, estimation
)

In [ ]:
df_metrics = pd.DataFrame()

control_alpha = 0.0
control_arm = "reg20_geosort"
test_alpha = 0.0
test_arm = "variationA"
estimation = "rolling"
df_metrics = df_metrics.append(
    get_impression_weighted_avg(
        df_full_geo_metrics,
        df_visits,
        control_alpha,
        test_alpha,
        control_arm,
        test_arm,
        estimation,
    )
)

In [ ]:
control_alpha = 0.0
control_arm = "reg20_geosort"
test_alpha = 0.5
test_arm = "variationA"
estimation = "rolling"
df_metrics = df_metrics.append(
    get_impression_weighted_avg(
        df_full_geo_metrics,
        df_visits,
        control_alpha,
        test_alpha,
        control_arm,
        test_arm,
        estimation,
    )
)

## Order weighted Aggregated difference

In [ ]:
region_order_weights = db_handler.get_region_order_weights(
        start_date,
        end_date,
        region_model='20region'
)

region_order_weights[
    "order_weights"
] = region_order_weights.order_count_by_region / np.sum(
    region_order_weights.order_count_by_region
)

df_metrics = pd.merge(df_metrics, region_order_weights, on="customer_region_id")


df_metrics_order_weighted = df_metrics.loc[
    :,
    [
        "customer_region_id",
        "grp1",
        "grp1_alpha",
        "grp2",
        "grp2_alpha",
        "shipping_values",
        "vcd_values",
        "shipping_savings_values",
        "cvr_scores",
        "cf_scores"
        "order_weights",
    ],
].sort_values(by=["grp1", "grp1_alpha", "grp2", "grp2_alpha", "customer_region_id"])
df_metrics_order_weighted["order_adjusted_cvr_diff"] = (
    df_metrics_order_weighted["cvr_scores"] * df_metrics_order_weighted["order_weights"]
)
df_metrics_order_weighted["order_adjusted_cf_scores"] = (
    df_metrics_order_weighted["cf_scores"] * df_metrics_order_weighted["order_weights"]
)
df_metrics_order_weighted["order_adjusted_shipping_value_diff"] = (
    df_metrics_order_weighted["shipping_values"]
    * df_metrics_order_weighted["order_weights"]
)
df_metrics_order_weighted["order_adjusted_vcd_diff"] = (
    df_metrics_order_weighted["vcd_values"] * df_metrics_order_weighted["order_weights"]
)
df_metrics_order_weighted["order_adjusted_shipping_savings_diff"] = (
    df_metrics_order_weighted["shipping_savings_values"] * df_metrics_order_weighted["order_weights"]
)

df_metrics_order_weighted[
    [
        "grp1",
        "grp1_alpha",
        "grp2",
        "grp2_alpha",
        "order_adjusted_cvr_diff",
        "order_adjusted_cf_scores",
        "order_adjusted_shipping_value_diff",
        "order_adjusted_vcd_diff",
        "order_adjusted_shipping_savings_diff"
    ]
].groupby(["grp1", "grp1_alpha", "grp2", "grp2_alpha"]).sum().reset_index()